In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
from sklearn.linear_model import LinearRegression

In [68]:
df = pd.read_csv('CGMData/CGM_with_features.csv')
display(df)

,Readings (CGM / BGM),DateTime,Hour,Minute,Eating,Activity Level,Carbohydrate Intake
0,153,2024-03-05 00:03:00,0,3,No,Low,0
1,161,2024-03-05 00:08:00,0,8,No,Low,0
2,171,2024-03-05 00:13:00,0,13,No,Low,0
3,185,2024-03-05 00:18:00,0,18,No,Low,0
4,198,2024-03-05 00:23:00,0,23,No,Low,0
...,...,...,...,...,...,...,...
3972,400,2024-03-18 23:36:00,23,36,No,Low,0
3973,400,2024-03-18 23:41:00,23,41,No,Low,0
3974,400,2024-03-18 23:46:00,23,46,No,Low,0
3975,400,2024-03-18 23:51:00,23,51,No,Low,0


In [67]:
df = df.dropna()

last_3_days_data = df[df['DateTime'] >= df['DateTime'].max() - timedelta(days=3)]

# Split the data into training and testing sets
X = last_3_days_data[['Hour', 'Minute']]
y = last_3_days_data['Readings (CGM / BGM)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Train the RandomForestRegression model
rf = RandomForestRegressor(n_estimators=100, random_state=0)
rf.fit(X_train, y_train)

# Make predictions for 30 minutes into the future
predictions = rf.predict(X_test)

# Calculate and print the Mean Squared Error (MSE)
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

# Example prediction for the next 30 minutes
next_hour = df['Hour'].iloc[-1]  # Assuming the last entry in the DataFrame is the latest hour
next_minute = df['Minute'].iloc[-1] + 30  # Adding 30 minutes to the latest minute
if next_minute >= 60:
    next_hour += 1
    next_minute -= 60
prediction_input = np.array([[next_hour, next_minute]])
next_prediction = rf.predict(prediction_input)
print(f'Predicted glucose reading for {next_hour:02d}:{next_minute:02d}: {next_prediction[0]}')

TypeError: unsupported operand type(s) for -: 'str' and 'datetime.timedelta'

In [64]:
from sklearn.preprocessing import OneHotEncoder


# Convert categorical features to one-hot encoding
encoder = OneHotEncoder(sparse=False, drop='first')  # drop='first' to avoid multicollinearity
encoded_features = encoder.fit_transform(X[['Eating', 'Activity Level']])
encoded_feature_names = encoder.get_feature_names(['Eating', 'Activity Level'])
X_encoded = pd.concat([X.drop(['Eating', 'Activity Level'], axis=1), pd.DataFrame(encoded_features, columns=encoded_feature_names)], axis=1)

# Split the encoded data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=0)

# Train the RandomForestRegression model
rf = RandomForestRegressor(n_estimators=100, random_state=0)
rf.fit(X_train, y_train)

# Make predictions for 30 minutes into the future
predictions = rf.predict(X_test)

# Calculate and print the Mean Squared Error (MSE)
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

# Example prediction for the next 30 minutes
next_hour = df['Hour'].iloc[-1]  # Assuming the last entry in the DataFrame is the latest hour
next_minute = df['Minute'].iloc[-1] + 30  # Adding 30 minutes to the latest minute
if next_minute >= 60:
    next_hour += 1
    next_minute -= 60
prediction_input = pd.DataFrame([[next_hour, next_minute, 'Yes', 50, 'High']], columns=['Hour', 'Minute', 'Eating', 'Carbohydrate Intake', 'Activity Level'])  # Example values for new features
prediction_input_encoded = pd.concat([prediction_input.drop(['Eating', 'Activity Level'], axis=1), pd.DataFrame(encoder.transform(prediction_input[['Eating', 'Activity Level']]), columns=encoded_feature_names)], axis=1)
next_prediction = rf.predict(prediction_input_encoded)
print(f'Predicted glucose reading for {next_hour:02d}:{next_minute:02d}: {next_prediction[0]}')

c:\Users\zar10\miniconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


AttributeError: 'OneHotEncoder' object has no attribute 'get_feature_names'